In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import time

BASE_URL = "https://openpath.kr/knowhow/open-wiki?category=&text=&page={page}"

DOMAIN = "Design"
DEPT_ID = "003"

headers = {
    "User-Agent": "OpenpathGlossaryCrawler/1.0 (contact: your-email@example.com)"
}

def fetch_page_html(page: int) -> str:
    url = BASE_URL.format(page=page)
    resp = requests.get(url, headers=headers, timeout=10)
    resp.raise_for_status()
    return resp.text

def parse_terms(html: str):
    soup = BeautifulSoup(html, "html.parser")
    results = []

    # #contents 안의 li 들 중에서, h4 > span > span 과 p 가 같이 있는 것만 사용
    for li in soup.select("#contents li"):
        title_span = li.select_one("h4 span span")
        p_tag = li.select_one("p")
        if not title_span or not p_tag:
            continue

        title = title_span.get_text(strip=True)
        answer = p_tag.get_text(" ", strip=True)

        results.append({
            "title": title,
            "answer": answer,
            "domain": DOMAIN,
            "dept_id": DEPT_ID
        })
    return results

def crawl_all_pages(start_page=1, end_page=32):
    all_terms = []

    for page in range(start_page, end_page + 1):
        print(f"[페이지 {page}] 수집 중...")
        html = fetch_page_html(page)
        terms = parse_terms(html)
        print(f"  → {len(terms)}개 추출")

        all_terms.extend(terms)
        time.sleep(0.7) 

    return all_terms

if __name__ == "__main__":
    terms = crawl_all_pages(1, 32)

    print("총 수집 용어 수:", len(terms))

    # 전체 JSON 저장
    with open("openpath_glossary.json", "w", encoding="utf-8") as f:
        json.dump(terms, f, ensure_ascii=False, indent=2)


[페이지 1] 수집 중...
  → 0개 추출
[페이지 2] 수집 중...
  → 0개 추출
[페이지 3] 수집 중...
  → 0개 추출
[페이지 4] 수집 중...
  → 0개 추출
[페이지 5] 수집 중...
  → 0개 추출
[페이지 6] 수집 중...
  → 0개 추출
[페이지 7] 수집 중...
  → 0개 추출
[페이지 8] 수집 중...
  → 0개 추출
[페이지 9] 수집 중...
  → 0개 추출
[페이지 10] 수집 중...
  → 0개 추출
[페이지 11] 수집 중...
  → 0개 추출
[페이지 12] 수집 중...
  → 0개 추출
[페이지 13] 수집 중...
  → 0개 추출
[페이지 14] 수집 중...
  → 0개 추출
[페이지 15] 수집 중...
  → 0개 추출
[페이지 16] 수집 중...
  → 0개 추출
[페이지 17] 수집 중...
  → 0개 추출
[페이지 18] 수집 중...
  → 0개 추출
[페이지 19] 수집 중...
  → 0개 추출
[페이지 20] 수집 중...
  → 0개 추출
[페이지 21] 수집 중...
  → 0개 추출
[페이지 22] 수집 중...
  → 0개 추출
[페이지 23] 수집 중...
  → 0개 추출
[페이지 24] 수집 중...
  → 0개 추출
[페이지 25] 수집 중...
  → 0개 추출
[페이지 26] 수집 중...
  → 0개 추출
[페이지 27] 수집 중...
  → 0개 추출
[페이지 28] 수집 중...
  → 0개 추출
[페이지 29] 수집 중...
  → 0개 추출
[페이지 30] 수집 중...
  → 0개 추출
[페이지 31] 수집 중...
  → 0개 추출
[페이지 32] 수집 중...
  → 0개 추출
총 수집 용어 수: 0


In [2]:
import time
import json

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException

BASE_URL = "https://openpath.kr/knowhow/open-wiki?category=&text=&page={page}"

# 필요에 따라 수정
DOMAIN = "Design"
DEPT_ID = "003"


def create_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # 화면 보고 싶으면 이 줄 주석 처리
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
                              options=options)
    return driver


def crawl_one_page(driver, page: int):
    url = BASE_URL.format(page=page)
    print(f"[page {page}] GET {url}")
    driver.get(url)
    time.sleep(1.5)  # 페이지 로딩 대기 (필요시 조정)

    results = []

    # li 전체: //*[@id="contents"]/div/section/div[2]/div/div[1]/ul/li
    li_xpath = '//*[@id="contents"]/div/section/div[2]/div/div[1]/ul/li'
    li_elements = driver.find_elements(By.XPATH, li_xpath)
    print(f"  li 개수: {len(li_elements)}")

    for idx, li in enumerate(li_elements, start=1):
        try:
            # 제목 span: .//h4/span/span
            title_el = li.find_element(By.XPATH, './/h4/span/span')
            title = title_el.text.strip()

            # 설명 p: .//p
            p_el = li.find_element(By.XPATH, './/p')
            answer = p_el.text.strip()

            if not title or not answer:
                continue

            results.append({
                "title": title,
                "answer": answer,
                "domain": DOMAIN,
                "dept_id": DEPT_ID
            })

        except NoSuchElementException:
            # 제목 또는 p가 없으면 스킵
            print(f"  li {idx}: 필요한 요소 없음 → skip")
            continue
        except Exception as e:
            print(f"  li {idx}: 예외 발생 → {e}")
            continue

    return results


def crawl_all_pages(start_page=1, end_page=32):
    driver = create_driver()
    all_terms = []

    try:
        for page in range(start_page, end_page + 1):
            page_terms = crawl_one_page(driver, page)
            print(f"  → page {page}: {len(page_terms)} 개 추출")
            all_terms.extend(page_terms)
            time.sleep(0.7)  # 서버 보호용 딜레이

    finally:
        driver.quit()

    return all_terms


if __name__ == "__main__":
    terms = crawl_all_pages(1, 32)
    print("\n총 수집 개수:", len(terms))

    with open("openpath_glossary_selenium.json", "w", encoding="utf-8") as f:
        json.dump(terms, f, ensure_ascii=False, indent=2)

    print("파일 저장 완료: openpath_glossary_selenium.json")


[page 1] GET https://openpath.kr/knowhow/open-wiki?category=&text=&page=1
  li 개수: 10
  → page 1: 10 개 추출
[page 2] GET https://openpath.kr/knowhow/open-wiki?category=&text=&page=2
  li 개수: 10
  → page 2: 10 개 추출
[page 3] GET https://openpath.kr/knowhow/open-wiki?category=&text=&page=3
  li 개수: 10
  → page 3: 10 개 추출
[page 4] GET https://openpath.kr/knowhow/open-wiki?category=&text=&page=4
  li 개수: 10
  → page 4: 10 개 추출
[page 5] GET https://openpath.kr/knowhow/open-wiki?category=&text=&page=5
  li 개수: 10
  → page 5: 10 개 추출
[page 6] GET https://openpath.kr/knowhow/open-wiki?category=&text=&page=6
  li 개수: 10
  → page 6: 10 개 추출
[page 7] GET https://openpath.kr/knowhow/open-wiki?category=&text=&page=7
  li 개수: 10
  → page 7: 10 개 추출
[page 8] GET https://openpath.kr/knowhow/open-wiki?category=&text=&page=8
  li 개수: 10
  → page 8: 10 개 추출
[page 9] GET https://openpath.kr/knowhow/open-wiki?category=&text=&page=9
  li 개수: 10
  → page 9: 10 개 추출
[page 10] GET https://openpath.kr/knowhow/open

In [3]:
import re
import time
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


BASE_LIST_URL = "https://www.gokams.or.kr/visual-art/art-terms/glossary/art_list.asp"
BASE_DETAIL_URL = "https://www.gokams.or.kr/visual-art/art-terms/glossary/art_view.asp?idx={idx}&page=1&s1=&s2=&flag_initial="

DOMAIN = "Design"
DEPT_ID = "003"

OUTPUT_FILE = "art_terms.jsonl"


def create_driver():
    options = webdriver.ChromeOptions()
    # options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    return webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )


def click_more_until_end(driver):
    """더보기 버튼이 사라질 때까지 클릭"""
    while True:
        try:
            btn = driver.find_element(By.XPATH, '//*[@id="btn_more"]')
            if btn.is_displayed():
                btn.click()
                time.sleep(1)
            else:
                break
        except:
            break


def extract_list_items(driver):
    """목록의 title 과 idx 추출"""

    items = driver.find_elements(By.XPATH, '//*[@id="html_data_list"]/tr/td[1]/a')
    results = []

    for a in items:
        title = a.text.strip()
        href = a.get_attribute("href")

        match = re.search(r"go_view\('(\d+)'\)", href)
        if not match:
            continue

        idx = match.group(1)

        results.append({"title": title, "idx": idx})

    return results


def extract_answer(driver, idx, retry=3):
    """상세 페이지에서 answer 추출 (retry 포함)"""

    url = BASE_DETAIL_URL.format(idx=idx)

    for attempt in range(retry):
        try:
            driver.get(url)
            time.sleep(1)
            driver.execute_script("window.scrollTo(0, 500);")
            time.sleep(0.7)

            answer_el = driver.find_element(
                By.XPATH,
                '//*[@id="container"]/div[1]/div[1]/div[2]/table/tbody/tr[4]/td'
            )

            return answer_el.text.strip()

        except Exception as e:
            if attempt == retry - 1:
                print(f"[ERROR] idx={idx} 상세페이지 실패: {e}")
                return ""

            time.sleep(1)

    return ""


def save_jsonl(record):
    """한 건씩 JSONL 파일로 저장"""
    with open(OUTPUT_FILE, "a", encoding="utf-8") as f:
        f.write(json.dumps(record, ensure_ascii=False) + "\n")


def crawl():
    driver = create_driver()
    driver.get(BASE_LIST_URL)
    time.sleep(1.5)

    print("더보기 클릭 중...")
    click_more_until_end(driver)

    print("목록 추출 중...")
    list_items = extract_list_items(driver)
    total = len(list_items)

    print(f"총 용어 수: {total}개\n")

    for i, item in enumerate(list_items, start=1):

        print(f"[{i} / {total}] {item['title']} (idx={item['idx']})")

        answer = extract_answer(driver, item["idx"])

        record = {
            "title": item["title"],
            "answer": answer,
            "domain": DOMAIN,
            "dept_id": DEPT_ID
        }

        save_jsonl(record)

    driver.quit()
    print("\n모든 크롤링 완료!")
    print("저장 파일:", OUTPUT_FILE)


if __name__ == "__main__":
    crawl()


더보기 클릭 중...
목록 추출 중...
총 용어 수: 300개

[1 / 300] 6대가 (idx=3)
[2 / 300] 건칠 (idx=15)
[3 / 300] 걸개그림 (idx=16)
[4 / 300] 경성박람회 (idx=21)
[5 / 300] 공공미술 (idx=30)
[6 / 300] 구상 (idx=40)
[7 / 300] 구상조각 (idx=41)
[8 / 300] 극사실주의 (idx=49)
[9 / 300] 근대성 (idx=51)
[10 / 300] 기록화 (idx=58)
[11 / 300] 기명절지화 (idx=59)
[12 / 300] 길상화 (idx=62)
[13 / 300] 나전칠기 (idx=65)
[14 / 300] 낙관 (idx=66)
[15 / 300] 낙죽 (idx=68)
[16 / 300] 남화 (idx=70)
[17 / 300] 능묘조각 (idx=82)
[18 / 300] 단색화 (idx=89)
[19 / 300] 단청 (idx=91)
[20 / 300] 달항아리 (idx=94)
[21 / 300] 당초문 (idx=95)
[22 / 300] 대목장 (idx=99)
[23 / 300] 대안공간 (idx=100)
[24 / 300] 대지미술 (idx=103)
[25 / 300] 대한민국미술전람회 (idx=104)
[26 / 300] 도조 (idx=111)
[27 / 300] 동양화 (idx=120)
[28 / 300] 막사발 (idx=130)
[29 / 300] 매병 (idx=135)
[30 / 300] 모노크롬 (idx=142)
[31 / 300] 모더니즘 (idx=144)
[32 / 300] 목공예 (idx=148)
[33 / 300] 물성 (idx=168)
[34 / 300] 미니멀리즘 (idx=171)
[35 / 300] 미디어아트 (idx=174)
[36 / 300] 민중미술 (idx=192)
[37 / 300] 민화 (idx=193)
[38 / 300] 백묘화 (idx=210)
[39 / 300] 백자 (idx=211)
[40 

In [6]:
import json

input_file = 'gpt.jsonl'
output_file = 'gpt.json'

data = []

with open(input_file, 'r', encoding='utf-8') as f:
    for line in f:
        if line.strip():  # 빈 줄 제외
            data.append(json.loads(line))

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print("변환 완료:", output_file)

JSONDecodeError: Extra data: line 1 column 89 (char 88)

In [16]:
import time
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

MAIN_URL = "https://kli.korean.go.kr/term/indexMain.do?lang=kr"

DOMAIN = "Design"
DEPT_ID = "003"
OUTPUT_JSON = "kli_terms_426_3_all.json"

TOTAL_PAGES = 50      # 전체 페이지 수
BLOCK_SIZE = 5          # 1~5, 6~10, 11~15 ... 형식


def create_driver():
    options = webdriver.ChromeOptions()
    # 디버깅 시에는 주석 처리해서 브라우저 화면을 보면서 진행하는 것을 추천합니다.
    # options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )
    return driver


def open_search_result_list(driver):
    """메인 → lclas_name_426 → bubble_lst_426 li[3]/a 클릭 후 검색결과 리스트 진입"""
    driver.get(MAIN_URL)
    time.sleep(2)

    # 1. 대분류 클릭
    box = driver.find_element(By.ID, "lclas_name_426")
    box.find_element(By.TAG_NAME, "p").click()
    time.sleep(1)

    # 2. 버블 3번째 클릭
    driver.find_element(
        By.XPATH, '//*//*[@id="bubble_lst_426"]/li[4]/a'
    ).click()

    # 3. 리스트 페이지 로딩 대기
    WebDriverWait(driver, 10).until(
        EC.url_contains("indexSearchList.do")
    )
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "includeSearchList"))
    )
    time.sleep(1)
    print("검색 결과 리스트 페이지 진입 완료:", driver.current_url)


def parse_current_page(driver):
    """현재 페이지에서 title / answer 추출"""
    results = []

    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.XPATH, "//*[@id='includeSearchList']//strong[@class='kor']")
            )
        )
    except TimeoutException:
        print("  - strong.kor 요소가 안 보임 → 0개 처리")
        return results

    items = driver.find_elements(
        By.XPATH,
        "//*[@id='includeSearchList']//a[.//strong[@class='kor']]"
    )
    print(f"  - 항목 수: {len(items)}")

    for idx, a in enumerate(items, start=1):
        try:
            title_el = a.find_element(By.XPATH, ".//strong[@class='kor']")
            title = title_el.text.strip()

            answer_el = a.find_element(By.XPATH, ".//p[@class='txt']")
            answer = answer_el.text.strip()

            if not title and not answer:
                continue

            results.append({
                "title": title,
                "answer": answer,
                "domain": DOMAIN,
                "dept_id": DEPT_ID
            })
        except NoSuchElementException:
            print(f"    · {idx}번째 항목: title/answer 없음 → skip")
        except Exception as e:
            print(f"    · {idx}번째 항목 예외: {e} → skip")

    return results


def click_page_number(driver, page_num):
    """
    현재 블럭 안에서 'page_num'에 해당하는 페이지 번호 a 태그 클릭.
    - class는 num / num current active
    - onclick은 searchInfo.setPage(page_num)
    """
    page_str = str(page_num)

    # 페이지 변경 감지를 위한 현재 첫 title
    try:
        first_title_before = driver.find_element(
            By.XPATH, "//*[@id='includeSearchList']//strong[@class='kor']"
        ).text
    except Exception:
        first_title_before = None

    # 후보 XPATH 리스트 (텍스트 / onclick 기반)
    candidates = [
        f"//*[@id='includeSearchList']//a[normalize-space(text())='{page_str}']",
        f"//*[@id='includeSearchList']//a[contains(@onclick,'setPage({page_num})')]",
        f"//a[contains(@onclick,'setPage({page_num})')]",
    ]

    link = None
    for xp in candidates:
        try:
            link = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.XPATH, xp))
            )
            break
        except TimeoutException:
            continue

    if not link:
        print(f"  - 페이지 번호 {page_str} 링크를 찾지 못함")
        return False

    driver.execute_script("arguments[0].click();", link)

    # current active가 page_num이 될 때까지(or title 변경) 대기
    try:
        WebDriverWait(driver, 10).until(
            lambda d: (
                d.find_elements(
                    By.XPATH,
                    f"//*[@id='includeSearchList']//a[contains(@class,'current') and normalize-space(text())='{page_str}']"
                )
                or (
                    first_title_before is not None and
                    d.find_element(
                        By.XPATH, "//*[@id='includeSearchList']//strong[@class='kor']"
                    ).text != first_title_before
                )
            )
        )
    except Exception:
        time.sleep(1.5)

    return True


def click_next_block(driver):
    """
    '다음' 버튼 클릭 (5개 페이지 블럭 단위 이동)
    xpath 가 div[11]/div[12] 사이에서 바뀔 수 있으니 여러 후보 사용
    """
    candidate_xpaths = [
        "//*[@id='includeSearchList']/div[2]/div[12]/a[12]",
        "//*[@id='includeSearchList']/div[2]/div[11]/a[12]",
        "//*[@id='includeSearchList']//a[contains(text(),'다음')]",
    ]

    next_btn = None
    for xp in candidate_xpaths:
        try:
            next_btn = driver.find_element(By.XPATH, xp)
            break
        except NoSuchElementException:
            continue

    if not next_btn:
        print("  - '다음' 버튼을 찾지 못함 → 마지막 블럭으로 판단")
        return False

    driver.execute_script("arguments[0].click();", next_btn)
    time.sleep(1.5)
    return True


def crawl_block_pagination(total_pages=TOTAL_PAGES, block_size=BLOCK_SIZE, max_terms=None):
    """
    1~total_pages를 block_size(5) 단위로 크롤링.
    - 1~5 페이지 크롤링 → '다음' → 6~10 → '다음' → 11~15 → ...
    """
    driver = create_driver()
    open_search_result_list(driver)

    all_terms = []

    # 첫 블럭: 1 페이지는 이미 열려 있으므로 거기서 시작
    for block_start in range(1, total_pages + 1, block_size):
        block_end = min(block_start + block_size - 1, total_pages)
        print(f"\n===== 블럭 {block_start} ~ {block_end} 처리 =====")

        for page in range(block_start, block_end + 1):
            # 첫 블럭의 1페이지는 이미 열려 있으니 클릭 생략
            if not (block_start == 1 and page == 1):
                print(f"\n→ 페이지 {page} 이동 시도")
                moved = click_page_number(driver, page)
                if not moved:
                    print(f"페이지 {page} 이동 실패 → 여기서 종료")
                    driver.quit()
                    return all_terms

            print(f"===== 페이지 {page} 크롤링 =====")
            page_terms = parse_current_page(driver)
            print(f"  → {len(page_terms)}개 수집")
            all_terms.extend(page_terms)

            if max_terms is not None and len(all_terms) >= max_terms:
                print(f"\n수집 개수 {len(all_terms)}가 max_terms {max_terms} 도달 → 중단")
                driver.quit()
                return all_terms

        # 마지막 블럭이면 '다음' 누르지 않고 종료
        if block_end >= total_pages:
            print("\n모든 블럭 처리 완료")
            break

        print(f"\n블럭 {block_start} ~ {block_end} 완료 → '다음' 버튼으로 다음 블럭 이동")
        moved_block = click_next_block(driver)
        if not moved_block:
            print("다음 블럭으로 이동 실패 → 여기서 종료")
            break

    driver.quit()
    return all_terms


if __name__ == "__main__":
    # 먼저 1~20 페이지 정도만 테스트:
    # terms = crawl_block_pagination(total_pages=20, block_size=5)

    # 전체 1214페이지 크롤링:
    terms = crawl_block_pagination(total_pages=TOTAL_PAGES, block_size=BLOCK_SIZE)

    print(f"\n총 수집 용어 수: {len(terms)}")

    with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
        json.dump(terms, f, ensure_ascii=False, indent=2)

    print("저장 완료 →", OUTPUT_JSON)


검색 결과 리스트 페이지 진입 완료: https://kli.korean.go.kr/term/search/indexSearchList.do?lang=kr

===== 블럭 1 ~ 5 처리 =====
===== 페이지 1 크롤링 =====
  - 항목 수: 10
  → 10개 수집

→ 페이지 2 이동 시도
===== 페이지 2 크롤링 =====
  - 항목 수: 10
  → 10개 수집

→ 페이지 3 이동 시도
===== 페이지 3 크롤링 =====
  - 항목 수: 10
  → 10개 수집

→ 페이지 4 이동 시도
===== 페이지 4 크롤링 =====
  - 항목 수: 10
  → 10개 수집

→ 페이지 5 이동 시도
===== 페이지 5 크롤링 =====
  - 항목 수: 10
  → 10개 수집

블럭 1 ~ 5 완료 → '다음' 버튼으로 다음 블럭 이동

===== 블럭 6 ~ 10 처리 =====

→ 페이지 6 이동 시도
===== 페이지 6 크롤링 =====
  - 항목 수: 10
  → 10개 수집

→ 페이지 7 이동 시도
===== 페이지 7 크롤링 =====
  - 항목 수: 10
  → 10개 수집

→ 페이지 8 이동 시도
===== 페이지 8 크롤링 =====
  - 항목 수: 10
    · 4번째 항목: title/answer 없음 → skip
  → 9개 수집

→ 페이지 9 이동 시도
===== 페이지 9 크롤링 =====
  - 항목 수: 10
  → 10개 수집

→ 페이지 10 이동 시도
===== 페이지 10 크롤링 =====
  - 항목 수: 10
  → 10개 수집

블럭 6 ~ 10 완료 → '다음' 버튼으로 다음 블럭 이동

===== 블럭 11 ~ 15 처리 =====

→ 페이지 11 이동 시도
===== 페이지 11 크롤링 =====
  - 항목 수: 10
  → 10개 수집

→ 페이지 12 이동 시도
===== 페이지 12 크롤링 =====
  - 항목 수: 10
  → 10개 수집

→ 페이지 13